<a href="https://colab.research.google.com/github/Akshitha181203/PortfolioOptimization/blob/PPO/ipynbs/transformer_ppo_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Init

In [1]:
! pip install setuptools==65.5.0
! pip install -q stable_baselines3
! pip install -q PyPortfolioOpt
! pip install -q yahoo-finance
! pip install -q yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install setuptools==65.5.0
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 34.1 MB/s eta 0:00:00
  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-el5q3qal
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-el5q3qal
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 69776b349ee4e63efe3826f318aef8e5c5f59648
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-bqsd_k9f/elegantrl_55e2851b13da4d4bbda949a98a214b75
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-bqsd_k9f/elegantrl_55e2851b13da4d4bbda949a98a214b75
  Resolved https://github.com/AI4Finance-Foundation/Eleg

In [3]:
!pip install pandas_market_calendars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.4/198.4 kB 9.5 MB/s eta 0:00:00


In [4]:
!pip install --quiet "shimmy>=2.0"

In [5]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/usr/local/lib/python3.11/dist-packages/pandas_datareader/compat/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  PANDAS_VERSION = LooseVersion(pd.__version__)
/usr/local/lib/python3.11/dist-packages/pyfolio/pos.py:25: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [6]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

In [7]:
import sys
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
import copy
from typing import Any, Dict, List, Optional, Tuple, Type, TypeVar, Union
import torch as th
from torch.nn import functional as F
from stable_baselines3 import PPO,SAC
from stable_baselines3.common.buffers import ReplayBuffer
from stable_baselines3.common.noise import ActionNoise
from stable_baselines3.common.off_policy_algorithm import OffPolicyAlgorithm
from stable_baselines3.common.policies import BasePolicy
from stable_baselines3.common.type_aliases import GymEnv, MaybeCallback, Schedule
from stable_baselines3.common.utils import get_parameters_by_name, polyak_update
from stable_baselines3.sac.policies import CnnPolicy, MlpPolicy, MultiInputPolicy, SACPolicy
import yfinance as yf
from itertools import count
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from pandas_datareader import data as pdr
import numpy as np
import random
import math
import time
import gym
from scipy.special import softmax
from gym.utils import seeding
from gym import spaces
from stable_baselines3.common.vec_env import DummyVecEnv
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt import objective_functions
from pypfopt.efficient_frontier import EfficientFrontier
import torch
from collections import namedtuple, deque
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Dataset

In [8]:
TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-10-01'
TRADE_START_DATE = '2021-10-01'
TRADE_END_DATE = '2023-03-01'

In [9]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (97013, 8)


In [10]:
df.sort_values(['date','tic'],ignore_index=True).head()

Price,date,close,high,low,open,volume,tic,day
0,2010-01-04,6.440331,7.660714,7.585000,7.622500,493729600,AAPL,0
1,2010-01-04,40.265984,57.869999,56.560001,56.630001,5277400,AMGN,0
2,2010-01-04,32.720367,41.099998,40.389999,40.810001,6894300,AXP,0
3,2010-01-04,43.777550,56.389999,54.799999,55.720001,6186700,BA,0
4,2010-01-04,39.547817,59.189999,57.509998,57.650002,7325600,CAT,0


## Preprocess

In [11]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators


[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (3310, 8)
Successfully added vix
Successfully added turbulence index


In [12]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [13]:
mvo_df = processed_full.sort_values(['date','tic'],ignore_index=True)[['date','tic','close']]

In [14]:
len(processed),len(processed_full),len(df)

(95990, 95990, 97013)

In [15]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

85753
10237


In [16]:
train.tail()

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
2956,2021-09-30,UNH,371.414124,403.489990,390.459991,401.489990,3779900.0,3.0,-4.215708,406.339770,374.984614,40.895387,-222.645245,41.980385,393.482303,392.988886,23.139999,24.871754
2956,2021-09-30,V,216.924561,228.789993,222.630005,227.580002,7128500.0,3.0,-1.515440,225.179240,213.252813,44.079001,-54.448997,19.569853,221.375697,227.976408,23.139999,24.871754
2956,2021-09-30,VZ,42.742085,54.509998,54.000000,54.500000,18736600.0,3.0,-0.203776,43.718478,42.508789,41.824937,-101.984683,21.682953,43.259642,43.717979,23.139999,24.871754
2956,2021-09-30,WBA,38.408337,48.930000,46.919998,48.790001,6449400.0,3.0,-0.221534,42.405396,38.093747,44.613712,-106.145452,0.941150,40.094060,39.195162,23.139999,24.871754
2956,2021-09-30,WMT,44.220188,47.243332,46.416668,46.880001,22457700.0,3.0,-0.505126,47.482040,44.063715,40.165811,-151.472464,45.479205,46.244331,45.887429,23.139999,24.871754


In [17]:
trade.head()

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2021-10-01,AAPL,139.992538,142.919998,139.110001,141.899994,94639600.0,4.0,-1.686476,153.831505,135.763061,46.927792,-142.248873,41.749873,145.702434,144.809058,21.1,120.123128
0,2021-10-01,AMGN,191.379196,214.610001,210.800003,213.589996,2629400.0,4.0,-2.907907,199.755653,187.186060,40.408548,-97.158089,36.189244,195.730694,203.825873,21.1,120.123128
0,2021-10-01,AXP,165.851685,175.119995,168.479996,168.500000,3956000.0,4.0,2.217004,169.902251,145.535356,56.265126,117.355578,15.667511,157.221232,159.408877,21.1,120.123128
0,2021-10-01,BA,226.000000,226.720001,220.600006,222.850006,9113600.0,4.0,0.730320,226.909442,205.727561,51.614047,116.649440,2.027170,217.175334,221.968500,21.1,120.123128
0,2021-10-01,CAT,180.580658,195.869995,191.240005,192.899994,3695500.0,4.0,-3.498002,197.692325,174.339346,41.999419,-112.225084,36.203176,189.292053,192.682352,21.1,120.123128


In [18]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [19]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 291


In [20]:
train

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2010-01-04,AAPL,6.440331,7.660714,7.585000,7.622500,493729600.0,0.0,0.000000,6.461650,6.430150,100.000000,66.666667,100.000000,6.440331,6.440331,20.040001,0.000000
0,2010-01-04,AMGN,40.265984,57.869999,56.560001,56.630001,5277400.0,0.0,0.000000,6.461650,6.430150,100.000000,66.666667,100.000000,40.265984,40.265984,20.040001,0.000000
0,2010-01-04,AXP,32.720367,41.099998,40.389999,40.810001,6894300.0,0.0,0.000000,6.461650,6.430150,100.000000,66.666667,100.000000,32.720367,32.720367,20.040001,0.000000
0,2010-01-04,BA,43.777550,56.389999,54.799999,55.720001,6186700.0,0.0,0.000000,6.461650,6.430150,100.000000,66.666667,100.000000,43.777550,43.777550,20.040001,0.000000
0,2010-01-04,CAT,39.547817,59.189999,57.509998,57.650002,7325600.0,0.0,0.000000,6.461650,6.430150,100.000000,66.666667,100.000000,39.547817,39.547817,20.040001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2956,2021-09-30,UNH,371.414124,403.489990,390.459991,401.489990,3779900.0,3.0,-4.215708,406.339770,374.984614,40.895387,-222.645245,41.980385,393.482303,392.988886,23.139999,24.871754
2956,2021-09-30,V,216.924561,228.789993,222.630005,227.580002,7128500.0,3.0,-1.515440,225.179240,213.252813,44.079001,-54.448997,19.569853,221.375697,227.976408,23.139999,24.871754
2956,2021-09-30,VZ,42.742085,54.509998,54.000000,54.500000,18736600.0,3.0,-0.203776,43.718478,42.508789,41.824937,-101.984683,21.682953,43.259642,43.717979,23.139999,24.871754
2956,2021-09-30,WBA,38.408337,48.930000,46.919998,48.790001,6449400.0,3.0,-0.221534,42.405396,38.093747,44.613712,-106.145452,0.941150,40.094060,39.195162,23.139999,24.871754


# Environment

In [21]:
x_train = []
x_trade = []

In [22]:
for d in train.date.unique():
  x_train.append(train[train.date == d].iloc[:,5:].to_numpy())
for d in trade.date.unique():
  x_trade.append(trade[trade.date==d].iloc[:,5:].to_numpy())

In [23]:
x_train = np.array(x_train)
x_trade = np.array(x_trade)

In [24]:
x_train.shape,x_trade.shape

((2957, 29, 13), (353, 29, 13))

vars
* window_size
* data
* day
* current_step
* action_space = 1D (0 to 1)
* observation_space = 3D (window_size, stock_index, closing and other paramters)
* portfolio_value = 1  

In [25]:
class PortfolioEnv(gym.Env):
    def __init__(self, data, window_size, reward_type='sharpe'):
        super(PortfolioEnv, self).__init__()
        self.data = data
        self.window_size = window_size

        self.day = window_size
        self.reward_type = reward_type
        self.returns_memory = []
        self.current_step = window_size
        # Define the action and observation space
        self.action_space = spaces.Box(low=-20, high=20, shape=(self.data.shape[1],), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.window_size, self.data.shape[1], self.data.shape[2]+1), dtype=np.float32)

        self.portfolio_value = 1e6
        self.balance = 1e6

        self.weights = np.zeros(self.data.shape[1])
        self.weights.fill(1/self.data.shape[1])
        self.memory = []
        self.weights_memory = []
        # Initialize the state
        self.reset()

    def step(self, action):
        # Execute one step of the environment
        action = softmax(action) # Clip action values to [0, 1]
        self.weights = np.array(action).reshape(-1)
        self.weights_memory.append(self.weights)

        self.portfolio_value = self.balance
        self.memory.append(self.portfolio_value)

        # Update the state
        self.observation = self.get_observation()
        # reward = 0
        # for i in range(self.data.shape[1]):
        #     reward += ((self.data[self.day+1,i,0]-self.data[self.day,i,0])/self.data[self.day,i,0])*self.weights[i]*self.balance
        # self.balance += reward

        step_return = 0
        for i in range(self.data.shape[1]):
            r = ((self.data[self.day+1, i, 0] - self.data[self.day, i, 0]) / self.data[self.day, i, 0]) * self.weights[i]
            step_return += r
        self.returns_memory.append(step_return)
        self.balance += step_return * self.balance

        # Move to the next time step
        self.current_step += 1
        self.day += 1
        # Calculate the reward and done flag
        done = self.day >= self.data.shape[0] - 1
        # return self.observation, (reward/1e6), done, {}
        if done:
            if self.reward_type == 'sortino':
                reward = self.sortino_ratio(self.returns_memory)
            elif self.reward_type == 'sharpe':
                reward = self.sharpe_ratio(self.returns_memory)
            else:
                reward = sum(self.returns_memory)
        else:
            reward = step_return

        return self.observation, reward, done, {}

    def sharpe_ratio(self, returns, risk_free_rate=0.0):
        excess = np.array(returns) - risk_free_rate
        return np.mean(excess) / (np.std(excess) + 1e-6)

    def sortino_ratio(self, returns, risk_free_rate=0.0):
        downside = np.array([r for r in returns if r < risk_free_rate])
        return (np.mean(returns) - risk_free_rate) / (np.std(downside) + 1e-6 if len(downside) > 0 else 1e-6)

    def reset(self):
        # Reset the environment to the initial state
        self.current_step = self.window_size
        self.day = self.window_size
        self.portfolio_value = 1e6
        self.balance = 1e6
        self.weights = np.zeros(self.data.shape[1])
        self.weights.fill(1/self.data.shape[1])
        self.memory = []
        self.weights_memory = []
        # Initialize the state
        self.observation = self.get_observation()

        return self.observation

    def get_observation(self):
        state = np.zeros(self.observation_space.shape)
        state[:,:,:-1] = np.array(self.data[self.day-self.window_size:self.day])
        state[:,:,-1] = np.repeat(self.weights.reshape(1,-1),self.observation_space.shape[0],axis=0)
        return state

# Transformer Policy

In [26]:
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import torch
import numpy as np

from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.distributions import SquashedDiagGaussianDistribution
from stable_baselines3.common.policies import ActorCriticPolicy

from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

In [27]:
class TransformerEncoderExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, nhead=7, num_layers=4, dim_feedforward=1024,hidden_units=1024):
        super(TransformerEncoderExtractor, self).__init__(observation_space, features_dim=1024)

        input_shape = observation_space.shape # Assumes the input is a tensor of shape (batch_size, time, stocks, features)

        self.encoder_layer = TransformerEncoderLayer(d_model=406, nhead=nhead, dim_feedforward=dim_feedforward,norm_first=True)
        self.transformer_encoder = TransformerEncoder(encoder_layer=self.encoder_layer, num_layers=num_layers)
        self.dense = nn.Linear(input_shape[0]*input_shape[1]*input_shape[2], hidden_units)

    def forward(self, observations):
        # observations has shape (batch_size, time, stocks, features)
        batch_size, time, stocks, features = observations.shape
        x = observations.transpose(0, 1)
        x = x.view(time, batch_size, stocks * features)
        x = self.transformer_encoder(x)
        x = x.reshape(batch_size,-1)
        x = self.dense(x)
        # x = ( x - torch.min(x) )/ (torch.max(x) - torch.min(x))
        return x


# Train


In [28]:
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

In [29]:
%cd /content

/content


In [30]:
env = PortfolioEnv(x_train, 7, reward_type="sharpe")  # "sharpe" or "sortino"

In [31]:
x_train.shape

(2957, 29, 13)

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
stock_dim = env.action_space.shape[0]
policy_kwargs = dict(
    features_extractor_class=TransformerEncoderExtractor,
    features_extractor_kwargs=dict(),
)
#turn on for training
model = PPO("MlpPolicy", env,verbose=1,policy_kwargs=policy_kwargs,batch_size=8,learning_rate=0.003)
model.learn(total_timesteps=50000, progress_bar=False,log_interval=1)
model.save(f"/content/drive/MyDrive/Projects/ddpg_transformer_portfolio_{datetime.now().strftime('%d_%m_%Y_%H_%M_%S')}")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 78   |
|    iterations      | 1    |
|    time_elapsed    | 26   |
|    total_timesteps | 2048 |
-----------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 2.95e+03  |
|    ep_rew_mean          | 1.77      |
| time/                   |           |
|    fps                  | 6         |
|    iterations           | 2         |
|    time_elapsed         | 648       |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 2.0169027 |
|    clip_fraction        | 0.942     |
|    clip_range           | 0.2       |
|    entropy_loss         | -42.7     |
|    explained_variance   | -15.1     |
|    learning_rate        | 0.003     |
|    loss                 | 0.188     |
|    n_updates            | 10        |
|    policy_gradient_loss | 0.439     |
|    std                  | 1.

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
model = PPO.load('/content/drive/MyDrive/Projects/ddpg_transformer_portfolio_08_05_2023_17_18_46.zip',env)
#evaluate_policy(model, env, n_eval_episodes=1)

In [ ]:
trade_env = PortfolioEnv(x_trade, 7, reward_type="sharpe")  # "sharpe" or "sortino"

In [ ]:
portfolio_values = []
portfolio_actions = []

In [ ]:
obs = trade_env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = trade_env.step(action)
    if(dones==True):
      portfolio_values = trade_env.memory
      portfolio_actions = trade_env.weights_memory
      break

In [ ]:
portfolio_values = np.array(portfolio_values)

In [ ]:
import seaborn as sns
sns.set_theme(style="darkgrid")

In [ ]:
x = np.asarray(trade.date.unique()[8:], dtype='datetime64[s]')
plt.plot(x,portfolio_values/portfolio_values[0],c='r',label='PPO-Transformer')
plt.plot(x,x_trade[8:,:,0].sum(axis=1)/x_trade[8:,:,0].sum(axis=1)[0],label='DJI')
plt.gcf().autofmt_xdate()
plt.legend()
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.title('Cumulative Returns during Recession')
plt.show()

In [ ]:
portfolio_df = pd.DataFrame({'date':x,'account_value':portfolio_values})
portfolio_df.to_csv('ppo_trans_returns.csv')

In [ ]:
sns.reset_orig()
plt.stackplot(np.arange(345),np.array(portfolio_actions).T)
plt.show()

In [ ]:
model.predict(obs)

In [ ]:
softmax(model.predict(obs)[0])

In [ ]:
np.expand_dims(obs, axis=0).shape

In [ ]:
a = x_trade[:,:,0]
plt.plot(np.diff(a, axis=0) / a[:-1] * 100)
plt.show()

In [ ]:
model.feature_extractor

In [ ]:
env_sortino = PortfolioEnv(x_train, 7, reward_type="sortino")
model_sortino = PPO("MlpPolicy", env_sortino, ...)
model_sortino.learn(...)
model_sortino.save("/content/drive/...ppo_sortino.zip")

# Evaluate
trade_env_sortino = PortfolioEnv(x_trade, 7, reward_type="sortino")
obs = trade_env_sortino.reset()
while True:
    action, _ = model_sortino.predict(obs)
    obs, rewards, done, _ = trade_env_sortino.step(action)
    if done:
        portfolio_values_sortino = trade_env_sortino.memory
        break

In [ ]:
x = np.asarray(trade.date.unique()[8:], dtype='datetime64[s]')

plt.plot(x, np.array(portfolio_values_sharpe)/portfolio_values_sharpe[0], label="PPO-Sharpe")
plt.plot(x, np.array(portfolio_values_sortino)/portfolio_values_sortino[0], label="PPO-Sortino")
plt.plot(x, x_trade[8:,:,0].sum(axis=1)/x_trade[8:,:,0].sum(axis=1)[0], label='DJI')
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.title("Comparison: Sharpe vs Sortino PPO Agent")
plt.legend()
plt.gcf().autofmt_xdate()
plt.show()